#####  环境：阿里云 ubuntu  python27 tensorflowf1.4.0

In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'1.4.0'

## 5.1 MNIST 数据处理

#### 虽然MNIST数据集只提供了训练和测试数据，但是为了验证模型训练效果，一般会从训练数据中划分出一部分数据作为验证数据。

#### Tensorflow提供了一个类处理该数据集，它会自动下载并转化MNIST数据的格式，下面给出它的样例程序

In [5]:
# TensorFlow的MNIST类样例

from tensorflow.examples.tutorials.mnist import input_data

#载入数据，如果指定路径无该数据集，则会自动下载
mnist = input_data.read_data_sets("./data/mnist/",one_hot=True)

# 打印训练数据大小
print "Training data size:",mnist.train.num_examples

# 打印验证数据大小
print "Validating data size:",mnist.validation.num_examples

# 打印测试数据大小
print "Testing data size:",mnist.test.num_examples

Extracting ./data/mnist/train-images-idx3-ubyte.gz
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz
Training data size: 55000
Validating data size: 5000
Testing data size: 10000


In [6]:
# 打印训练数据示例
print "Training data example:",mnist.train.images[0]

Training data example: [0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.        

In [10]:
# 打印训练数据标签
print "Training data label:",mnist.train.labels[0]

Training data label: [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


![1356](./TfInActionImg/1356.PNG)

In [11]:
batch_size = 100
xs,ys = mnist.train.next_batch(batch_size)
print "X shape:",xs.shape
print "Y shape:",ys.shape

X shape: (100, 784)
Y shape: (100, 10)


In [14]:
print "Training data size:",mnist.train.images[55].shape

Training data size: (784,)


## 5.2 神经网络模型训练以及不同模型结果对比

### 5.2.1 TensorFlow训练神经网络

In [ ]:
# 运用第四章的方法，给出一个完整的神经网络程序
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# MNIST数据集相关的常数
INPUT_NODE = 784                    #输入层节点数，对于该数据集相当于图片像素28*28
OUTPUT_NODE = 10                    #输出层节点数，该数据集输出分为0~9类

#配置神经网络参数
LAYER1_NODE = 500                   #隐藏层节点数，用只有一个隐藏层的网络结构作为样例
BATCH_SIZE  = 100                   #数字越小，越接近随机梯度下降；越大越接近梯度下降
LEARNING_RATE_BASE = 0.8            #基础学习率
LEARNING_RATE_DECAY = 0.99          #学习率的衰减率
REGULARIZATION_RATE = 0.0001        #描述模型复杂程度的正则化项在损失函数中的系数
TRAINING_STEPS      = 30000         #训练轮数
MOVING_AVERAGE_DECAY = 0.99         #滑动平均衰减率

#定义一个使用relu激活函数的三层全连接神经网络
def inference(input_tensor,avg_class,weights1,biases1,weights2,biases2):
    #当没有使用滑动平均类的时候直接使用参数的当前取值
    if avg_class == None:
        # 计算隐藏层的前向传播结果
        layer1= tf.nn.relu(tf.matmul(input_tensor,weights1) + biases1)
        #计算输出层的前向传播结果。计算损失函数时候会一起计算softmax函数，所以这里无激活
        #函数。因为预测使用的是不同类别对应节点的输出值相对大小，有没有softmax层
        #对最后分类结果计算没有影响
        return tf.matmul(layer1,weights2) + biases2
    
    else:
        layer1 = tf.nn.relu(
            tf.matmul(input_tensor,avg_class.average(weights1)) + \
            avg_class.average(biases1)
        )
        return tf.matmul(layer1,avg_class.average(weights2)) + \
            avg_class.average(biases2)
    
# 训练模型的过程
def train(mnist):
    x = tf.placeholder(tf.float32,[None,INPUT_NODE],name = 'x-input')
    y_ = tf.placeholder(tf.float32,[None,OUTPUT_NODE],name = 'y-input')
    
    #生成隐藏层的参数
    weights1 = tf.Variable(tf.truncated_normal([INPUT_NODE,LAYER1_NODE],stddev=0.1))
    biases1 = tf.Variable(tf.constant(0.1,shape = [LAYER1_NODE]))
    
    #生成输出层参数
    weights2 = tf.Variable(tf.truncated_normal([LAYER1_NODE,OUTPUT_NODE],stddev=0.1))
    biases2 = tf.Variable(tf.constant(0.1,shape = [OUTPUT_NODE]))
    
    #计算如下参数下前向传播结果
    y = inference(x,None,weights1,biases1,weights2,biases2)
    
    #定义训练轮数的变量。这个变量不需要计算滑动平均值，所以指定变量为不可训练
    global_step = tf.Variable(0,trainable=False)
    
    #给定华东平均衰减率和训练轮数的变量，初始化华东平均类
    #？？因为给定训练轮数的变量可以加快训练早期变量的更新速度
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY,global_step)
    
    # 在代表神经网络参数的变量上使用滑动平均
    variables_averages_op = variable_averages.apply(tf.trainable_variables())
    
    # 计算滑动平均之后的前向传播结果
    average_y = inference(x,variable_averages,weights1,biases1,weights2,biases2)
    
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
    logits=y,labels=tf.argmax(y_,1))
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    
    #计算L2正则化损失函数
    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    regularization = regularizer(weights1) + regularizer(weights2)
    loss = cross_entropy_mean + regularization
    # 设置指数衰减的学习率
    
    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE,global_step,
        mnist.train.num_examples/BATCH_SIZE,LEARNING_RATE_DECAY
    )
    
    # 批梯度下降优化
    train_step = tf.train.GradientDescentOptimizer(learning_rate)\
    .minimize(loss,global_step = global_step)
    
    #反向传播更新神经网络参数，还要更新每个参数滑动平均值
    # train_op = tf.group(train_step,variables_averages_op)#同下面两行等价
    with tf.control_dependencies([train_step,variables_averages_op]):
        train_op = tf.no_op(name = "train")
    
    correct_prediction = tf.equal(tf.argmax(average_y,1),tf.arg_max(y_,1))
    #将布尔数值转换为实数数值，然后计算平均值即为一组数值上的正确率
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    
    # 初始化会话并开始训练过程
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        #准备验证数据
        #通过验证数据大致判断停止的条件和评判训练效果
        validate_feed = {x:mnist.validation.images,y_:mnist.validation.labels}
        
        #准备测试数据
        test_feed = {x:mnist.test.images,y_:mnist.test.labels}
        
        #迭代训练神经网络
        for i in range(TRAINING_STEPS):
            if i%1000 == 0:
                validate_acc = sess.run(accuracy,feed_dict = validate_feed)
                print("After %d training step(s), validation accuracy"
                     "using average model is %g"%(i,validate_acc))
            #产生这一轮使用的一个batch的数据，并运行训练过程
            xs,ys = mnist.train.next_batch(BATCH_SIZE)
            sess.run(train_op,feed_dict={x:xs,y_:ys})
        
        #在训练结束后，在测试数据上检测神经网络模型的最终正确率
        test_acc = sess.run(accuracy,feed_dict=test_feed)
        print("After %d training step(s), test accuracy using average"
                     "model is %g"%(TRAINING_STEPS,test_acc))
        
#主程序入口
def main(argv=None):
    mnist = input_data.read_data_sets("./data/mnist/",one_hot=True)
    train(mnist)
    
#Tensorflow提供一个主程序入口，tf.app.run会调用上面定义的main函数
if __name__ == '__main__':
    tf.app.run()

### 5.2.2 使用验证数据集判读模型效果

#### 为了估计模型在未知数据的效果，需要保证测试数据在训练过程不可见

#### 除了使用验证数据集，还可以使用交叉验证

### 5.2.3 不同模型效果比较

![1620](./TfInActionImg/1620.PNG)
![1626](./TfInActionImg/1626.PNG)

#### 从下图看出，只优化交叉熵损失虽然在训练数据上比优化总损失模型更小，但是在测试数据上的正确率却不好。

![1630](./TfInActionImg/1630.PNG)